## A1 - Banco de Dados
`Integrantes: Amanda Perez, Julia Queiroz e Kenner Azevedo`
<br>
`17 de setembro de 2021`

In [1]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import requests
import getpass

In [2]:
# dados gerados a partir do Fake Data Generator
df = pd.read_excel('books_database.xlsx')
df.head()

,autor,distribuidora,estado,preco,editora,edicao,titulo,cliente
0,Graham Q. Vasquez,Auctor Odio Corp.,Rio de Janeiro,47,Velit Aliquam Limited,9,Cum sociis natoque penatibus,Buckminster Hopkins
1,Melinda I. Carney,Nec Corporation,Rio de Janeiro,34,Dapibus Associates,5,In,Kevin Fischer
2,Callie Arnold,Pretium Neque Morbi LLP,Minas Gerais,87,Phasellus Dolor Elit Incorporated,8,Aliquam,Jerome Scott
3,Raya G. Singleton,Orci Phasellus Associates,São Paulo,16,Dapibus Ligula Aliquam Institute,9,Vivamus,Castor Battle
4,Carla I. Williamson,Phasellus In Company,São Paulo,72,Duis A Consulting,3,libero,Levi Noel


In [3]:
# adicionando gêneros de livros aleatoriamente ao dataframee
genero = ['Romance', 'Ficção', 'Terror', 'Suspense', 'Drama', 'Poesia']
df['genero'] = np.random.choice(genero, size = len(df))

In [4]:
# criando ids
for i in df.select_dtypes(exclude = [np.number]).columns:
    df['id_{}'.format(i)] = df.groupby([i]).ngroup()
    df['id_{}'.format(i)] = df['id_{}'.format(i)] + 1

In [5]:
# pegando UFs dos estados a partir da API do IBGE
url = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados'
r = requests.get(url)
ufs = pd.DataFrame(r.json())
ufs = ufs[['nome', 'sigla']]
df = pd.merge(df, ufs, how = 'left', left_on = 'estado', right_on = 'nome')
df.drop(columns = 'nome', inplace = True)

## Criando tabelas

In [6]:
df_Livros = df[['id_titulo', 'titulo', 'preco', 'edicao', 'id_editora', 'id_distribuidora']].sort_values(by = 'id_titulo', ascending = True).drop_duplicates(subset = 'id_titulo', keep = 'first')
df_Livros.rename(columns = {'id_titulo': 'id_livro', 'id_editora': 'Editora_id_editora', 'id_distribuidora': 'Distribuidora_id_distrib'}, inplace = True)

In [7]:
df_Rel_autor_livro = df[['id_titulo', 'id_autor']]
df_Rel_autor_livro.rename(columns = {'id_titulo': 'Livros_id_livro', 'id_autor': 'Autor_id_autor'}, inplace = True)
df_Rel_autor_livro['id_rel_aut_liv'] = df_Rel_autor_livro.groupby(['Livros_id_livro', 'Autor_id_autor']).ngroup()
df_Rel_autor_livro['id_rel_aut_liv'] = df_Rel_autor_livro['id_rel_aut_liv'] + 1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-7-6d70809bcf76>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Rel_autor_livro['id_rel_aut_liv'] = df_Rel_autor_livro.groupby(['Livros_id_livro', 'Autor_id_autor']).ngroup()
<ipython-input-7-6d70809bcf76>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

In [8]:
df_Autor = df[['id_autor', 'autor']].drop_duplicates(subset = 'id_autor', keep = 'first').sort_values(by = 'id_autor', ascending = True)
df_Autor.rename(columns = {'autor': 'nome'}, inplace = True)

In [9]:
df_Rel_genero_livro = df[['id_titulo', 'id_genero']]
df_Rel_genero_livro.rename(columns = {'id_titulo': 'Livros_id_livro', 'id_genero': 'Generos_id_genero'}, inplace = True)
df_Rel_genero_livro['id_rel_gen_liv'] = df_Rel_genero_livro.groupby(['Livros_id_livro', 'Generos_id_genero']).ngroup()
df_Rel_genero_livro['id_rel_gen_liv'] = df_Rel_genero_livro['id_rel_gen_liv'] + 1

<ipython-input-9-da14dffd51e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Rel_genero_livro['id_rel_gen_liv'] = df_Rel_genero_livro.groupby(['Livros_id_livro', 'Generos_id_genero']).ngroup()
<ipython-input-9-da14dffd51e6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Rel_genero_livro['id_rel_gen_liv'] = df_Rel_genero_livro['id_rel_gen_liv'] + 1


In [10]:
df_Genero = df[['id_genero', 'genero']].drop_duplicates(subset = 'id_genero', keep = 'first').sort_values(by = 'id_genero', ascending = True)

In [11]:
df_Estados_br = df[['id_estado', 'estado', 'sigla']].drop_duplicates(subset = 'id_estado', keep = 'first').sort_values(by = 'id_estado', ascending = True)
df_Estados_br.rename(columns = {'estado': 'nome', 'sigla': 'UF'}, inplace = True)

In [12]:
df_Cliente = df[['id_cliente', 'cliente']].drop_duplicates(subset = 'id_cliente', keep = 'first').sort_values(by = 'id_cliente', ascending = True)
df_Cliente.rename(columns = {'cliente': 'nome'}, inplace = True)

In [13]:
df_Rel_livro_cliente = df[['id_cliente', 'id_titulo']]
df_Rel_livro_cliente.rename(columns = {'id_titulo': 'Livros_id_livro', 'id_cliente': 'Cliente_id_cliente'}, inplace = True)
df_Rel_livro_cliente['id_compra'] = df_Rel_livro_cliente.groupby(['Livros_id_livro', 'Cliente_id_cliente']).ngroup()
df_Rel_livro_cliente['id_compra'] = df_Rel_livro_cliente['id_compra'] + 1

<ipython-input-13-104e8b5140ef>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Rel_livro_cliente['id_compra'] = df_Rel_livro_cliente.groupby(['Livros_id_livro', 'Cliente_id_cliente']).ngroup()
<ipython-input-13-104e8b5140ef>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Rel_livro_cliente['id_compra'] = df_Rel_livro_cliente['id_compra'] + 1


In [14]:
df_Editora = df[['id_editora', 'editora']].drop_duplicates(subset = 'id_editora', keep = 'first').sort_values(by = 'id_editora', ascending = True)
df_Editora.rename(columns = {'editora': 'nome'}, inplace = True)

In [15]:
df_Distribuidora = df[['id_distribuidora', 'distribuidora', 'id_estado']].drop_duplicates(subset = 'id_distribuidora', keep = 'first').sort_values(by = 'id_distribuidora', ascending = True)
df_Distribuidora.rename(columns = {'id_distribuidora': 'id_distrib', 'distribuidora': 'nome', 'id_estado': 'Estados_br_id_estado'}, inplace = True)

## Conectando à base de dados

In [16]:
p = getpass.getpass()

········


In [17]:
conn = pymysql.connect(host = '127.0.0.1', port = 3306, user = 'root', passwd = p)
cur = conn.cursor()

## Criando tabelas a partir de um arquivo SQL
O arquivo SQL foi extraído do Vertabelo, ambiente onde foi criado o modelo físico.

In [18]:
with open('create_databases.sql') as f:
    create_database_sql = f.read()
create_database_sql_list = create_database_sql.split(';')
create_database_sql_list = [q+';' for q in create_database_sql_list][:-1]

for query in create_database_sql_list:
    print(query)
    print(cur.execute(query))

-- Created by Vertabelo (http://vertabelo.com)
-- Last modification date: 2021-09-15 15:12:57.56

-- tables
-- Table: Autor
CREATE DATABASE IF NOT EXISTS Livraria;
1


USE Livraria;
0


CREATE TABLE Autor (
    id_autor int NOT NULL,
    nome varchar(35) NOT NULL,
    CONSTRAINT Autor_pk PRIMARY KEY (id_autor)
);
0


-- Table: Cliente
CREATE TABLE Cliente (
    id_cliente int NOT NULL,
    nome varchar(35) NOT NULL,
    CONSTRAINT Cliente_pk PRIMARY KEY (id_cliente)
);
0


-- Table: Distribuidora
CREATE TABLE Distribuidora (
    id_distrib int NOT NULL,
    nome varchar(40) NOT NULL,
    Estados_br_id_estado int NOT NULL,
    CONSTRAINT Distribuidora_pk PRIMARY KEY (id_distrib)
);
0


-- Table: Editora
CREATE TABLE Editora (
    id_editora int NOT NULL,
    nome varchar(40) NOT NULL,
    CONSTRAINT Editora_pk PRIMARY KEY (id_editora)
);
0


-- Table: Estados_br
CREATE TABLE Estados_br (
    id_estado int NOT NULL,
    nome varchar(30) NOT NULL,
    UF varchar(2) NOT NULL,
    CONSTRAIN

## Populando tabelas

In [19]:
cols = "`,`".join([str(i) for i in df_Autor.columns.tolist()])
for i, row in df_Autor.iterrows():
    sql = "INSERT INTO `Autor` (`" + cols + "`) VALUES (" + "%s,"*(len(row) - 1) + "%s)"
    cur.execute(sql, tuple(row))
    conn.commit()

In [20]:
cols = "`,`".join([str(i) for i in df_Genero.columns.tolist()])
for i, row in df_Genero.iterrows():
    sql = "INSERT INTO `Generos` (`" + cols + "`) VALUES (" + "%s,"*(len(row) - 1) + "%s)"
    cur.execute(sql, tuple(row))
    conn.commit()

In [21]:
cols = "`,`".join([str(i) for i in df_Estados_br.columns.tolist()])
for i, row in df_Estados_br.iterrows():
    sql = "INSERT INTO `Estados_br` (`" + cols + "`) VALUES (" + "%s,"*(len(row) - 1) + "%s)"
    cur.execute(sql, tuple(row))
    conn.commit()

In [22]:
cols = "`,`".join([str(i) for i in df_Editora.columns.tolist()])
for i, row in df_Editora.iterrows():
    sql = "INSERT INTO `Editora` (`" + cols + "`) VALUES (" + "%s,"*(len(row) - 1) + "%s)"
    cur.execute(sql, tuple(row))
    conn.commit()

In [23]:
cols = "`,`".join([str(i) for i in df_Cliente.columns.tolist()])
for i, row in df_Cliente.iterrows():
    sql = "INSERT INTO `Cliente` (`" + cols + "`) VALUES (" + "%s,"*(len(row) - 1) + "%s)"
    cur.execute(sql, tuple(row))
    conn.commit()

In [24]:
cols = "`,`".join([str(i) for i in df_Distribuidora.columns.tolist()])
for i, row in df_Distribuidora.iterrows():
    sql = "INSERT INTO `Distribuidora` (`" + cols + "`) VALUES (" + "%s,"*(len(row) - 1) + "%s)"
    cur.execute(sql, tuple(row))
    conn.commit()

In [25]:
cols = "`,`".join([str(i) for i in df_Livros.columns.tolist()])
for i, row in df_Livros.iterrows():
    sql = "INSERT INTO `Livros` (`" + cols + "`) VALUES (" + "%s,"*(len(row) - 1) + "%s)"
    cur.execute(sql, tuple(row))
    conn.commit()

In [26]:
cols = "`,`".join([str(i) for i in df_Rel_autor_livro.columns.tolist()])
for i, row in df_Rel_autor_livro.iterrows():
    sql = "INSERT INTO `Rel_autor_livro` (`" + cols + "`) VALUES (" + "%s,"*(len(row) - 1) + "%s)"
    cur.execute(sql, tuple(row))
    conn.commit()

In [27]:
cols = "`,`".join([str(i) for i in df_Rel_genero_livro.columns.tolist()])
for i, row in df_Rel_genero_livro.iterrows():
    sql = "INSERT INTO `Rel_genero_livro` (`" + cols + "`) VALUES (" + "%s,"*(len(row) - 1) + "%s)"
    cur.execute(sql, tuple(row))
    conn.commit()

In [28]:
cols = "`,`".join([str(i) for i in df_Rel_livro_cliente.columns.tolist()])
for i, row in df_Rel_livro_cliente.iterrows():
    sql = "INSERT INTO `Rel_livro_cliente` (`" + cols + "`) VALUES (" + "%s,"*(len(row) - 1) + "%s)"
    cur.execute(sql, tuple(row))
    conn.commit()